#  Sample script to apply a work item template on one or more work items

## Description 
- This script automates the process of applying a predefined work item template over one or more selected work items.
- It takes in a list of the selected work item's IDs, and the user entered work item template ID as input.
- It can update at most 1000 work items at a time. 
  
Note: To customize the automation logic, modify the [APIs](#APIs), [Algorithm](#Algorithm) and [Actions and output](#Actions-and-output) 

## Imports

 Import the necessary Python modules to execute the notebook. [**`nisystemlink.clients`**](https://github.com/ni/nisystemlink-clients-python) provides the predefined models and methods for `Work Items` and `Work Item Templates` APIs. **`Scrapbook`** is used to run notebooks and record data for integration with the SystemLink Notebook Execution Service.

In [ ]:
import scrapbook as sb
from typing import List, Optional

from nisystemlink.clients.work_item import WorkItemClient
from nisystemlink.clients.work_item.models import (
    QueryWorkItemsRequest,
    QueryWorkItemTemplatesRequest,
    UpdateWorkItemsRequest,
    UpdateWorkItemRequest,
    WorkItem,
    WorkItemTemplate
)

## Parameters

- **`work_item_ids`**: IDs of the work items to be updated.
- **`template_id`**: ID of the work item template used to update the work items.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
        "template_id": "",
        "work_item_ids": []
    }
  },
  "systemlink": {
    "parameters": [
        {
            "display_name": "Work item IDs",
            "id": "work_item_ids",
            "type": "string[]"
        },
        {
            "display_name": "Work item template ID",
            "id": "template_id",
            "type": "string"
        }
    ],
    "version": 1
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
# Add the work item IDs as a list of strings here
work_item_ids = []

# Add the work item template ID here
template_id = ""

## Python client

Initialize WorkItemClient to access work item and work item template APIs.

In [ ]:
work_item_client = WorkItemClient()

## APIs

Provides methods to query work items and work item templates by their IDs, and update work items.

Retrieve the work items by its IDs.

In [ ]:
def query_work_items(work_item_ids: List[str]) -> Optional[List[WorkItem]]:
    filter_conditions = ' || '.join(f'id == "{id}"' for id in work_item_ids)

    request = QueryWorkItemsRequest(
        filter=filter_conditions
    )

    try:
        response = work_item_client.query_work_items(request)
        return response.work_items
    except Exception as e:
        print(f"Error retrieving work items: {e}")
        return None

Retrieve the work item template by its ID.

In [ ]:
def get_work_item_template(template_id) -> Optional[WorkItemTemplate]:
    request = QueryWorkItemTemplatesRequest(
        filter = f'id == \"{template_id}\"'
    )

    try:
        response = work_item_client.query_work_item_templates(request)
    
        if response.work_item_templates == []:
            print(f"No template found with ID {template_id}")
            return None

        return response.work_item_templates[0]
    except Exception as e:
        print(f"Error retrieving template: {e}")
        return None

Updates the given work items and returns the IDs of successfully updated work items.

In [ ]:
def update_work_items(work_items) -> Optional[List[str]]:
    request = UpdateWorkItemsRequest(
         work_items = [UpdateWorkItemRequest(**work_item.model_dump()) for work_item in work_items],
         replace = True
     )

    try:
        work_items = work_item_client.update_work_items(request)
    except Exception as e:
        print(f"Error updating work items: {e}")
        return None

    updated_work_item_ids = [work_item.id for work_item in work_items.updated_work_items]

    return updated_work_item_ids

## Algorithm

Update the work items using the given work item template, and return the IDs of the updated work items.

In [ ]:
def update_work_items_with_template_data(work_items, work_item_template) -> Optional[List[str]]:
    for work_item in work_items:
        work_item.description = work_item_template.description
        work_item.test_program = work_item_template.test_program
        work_item.timeline.estimated_duration_in_seconds = work_item_template.timeline.estimated_duration_in_seconds
        work_item.resources.systems.filter = work_item_template.resources.systems.filter if work_item_template.resources.systems else None
        work_item.execution_actions = work_item_template.execution_actions
        work_item.properties = work_item_template.properties
        work_item.dashboard = work_item_template.dashboard
    
    updated_work_item_ids = update_work_items(work_items)
    return updated_work_item_ids

## Actions and output

Validates the inputs, manages the execution flow, prints the script output, and sends the output to the Execution Results page via Scrapbook. The output includes:

1. The total number of work items processed.
1. A list of updated work item IDs.
1. A list of work item IDs that were not updated.

In [ ]:
def update_work_items_with_template() -> None:
    updated_work_item_ids = []
    failed_work_item_ids = []

    if len(work_item_ids) == 0:
        print("Required atleast one work item id")
    elif len(work_item_ids) > 1000:
        print("Update limit exceeded: Only up to 1000 work items can be updated at a time.")
        sb.glue("Update limit exceeded: Only up to 1000 work items can be updated at a time.")
    elif not template_id:
        print("No template ID provided")
        sb.glue("Failed to fetch template", "No template ID provided")
    else:    
        # Fetch work item template
        work_item_template = get_work_item_template(template_id)
        
        if work_item_template is None:
            sb.glue("Failed to fetch template", template_id)
        else:
            # Fetch work items
            work_items = query_work_items(work_item_ids)
            if work_items is None:
                return
                
            # Update work items with template
            updated_work_item_ids = update_work_items_with_template_data(work_items, work_item_template)
            if updated_work_item_ids is None:
                return
    
            failed_work_item_ids = list(set(work_item_ids) - set(updated_work_item_ids))
    
            # Output
            print("Total work items:", len(work_item_ids))
            print("Work items updated:", ', '.join(updated_work_item_ids) if updated_work_item_ids else "--")
            print("Work items not updated:", ', '.join(failed_work_item_ids) if failed_work_item_ids else "--")
            
            # Executions page output
            sb.glue("Total work items:", len(work_item_ids))
            sb.glue("Work items updated:", ', '.join(updated_work_item_ids) if updated_work_item_ids else "--")
            sb.glue("Work items not updated:", ', '.join(failed_work_item_ids) if failed_work_item_ids else "--")

In [ ]:
update_work_items_with_template()

## Next Steps
Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as **`Work Item Automations`**.